## Z3 and Z3py resources

For this problem you will be solving constraint satisfaction and logic problems using the Z3 SMT solver via the z3py python library.

The demo code using z3py we went over in class is here: [link](https://drive.google.com/drive/folders/16HQXiwdcaman1IpC7H6vSJ71QQQ-aL2z?usp=sharing)

Online help for z3py is here: [link](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

## Installation

We first install to get the z3solver library using pip and test that works.

In [ ]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 27.7 MB/s eta 0:00:00


In [ ]:
# Run the first example from the z3py guide as a test: https://ericpony.github.io/z3py-tutorial/guide-examples.htm
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


# Sudoku as a constraint satisfaction problem (CSP)

Sudoku is a popular number-placement puzzle that originated in France in the end of the 19th century.  Modern Sudoku was likely invented by Howard Garns from Connersville, Indiana and was first published in 1979 under the name *Number Place*. The objective of the puzzle is to place numbers 1-9 on a 9$\times$9 grid, such that each number occurs only once in every row, every column, and every of the nine 3$\times$3 sub-grids that compose the main grid.
Sudoku puzzles are grids that have been partially occupied with numbers. The task is then to occupy the remaining fields in such a way that the constraints on rows, columns, and sub-grids are satisfied.
For more information about Sudoku refer to its Wikipedia page at http://en.wikipedia.org/wiki/Sudoku.

This problem has two parts. In the first part, you will write the boolean constraints in mathematical notation for solving a Sudoku puzzle. In the second part, you will write code and invoke the Z3 solver to solve the Sudoku instance and answer various questions about the solution.

## Part 1 (Constraints)
In text, define and write constraints over **boolean** variables corresponding to each number being in each cell. For example, you may use $X_{i,j}^k$ as teh variable that is true if and only if the number $k$ is in row $i$ and column $j$ (the variable is true if $k$ is in cell $(i, j)$ and false otherwise). Now write the following boolean constraints over these variables:

* Write the boolean formula for the constraints that the number 5 can occur at most once in the first row. **(3 point)**
* Write the boolean formula for the constraints that the number 6 can occur at most once in the first column. **(3 point)**
* Write the boolean formula for the constraints that the number 9 can occur at most once in the top left 3$\times$3 sub-grid. **(4 point)**

Solution:

1) for all n,m such that 1 <= n < m <= 9: ¬($X_{1,n}^5$ ∧ $X_{1,m}^5$)

2) for all n,m such that 1 <= n < m <= 9: ¬($X_{n,1}^6$ ∧ $X_{m,1}^6$)

3) for all (n1, m1) ≠ (n2, m2) with 1 ≤ n1, n2 ≤ 3 and 1 ≤ m1, m2 ≤ 3: ¬($X_{n1,m1}^9$ ∧ $X_{n2,m2}^9$)

All the constraints for the CSP Sudoky problem:

1.   Each cell is either empty or has one value in [1..9]
2.   Columns constraints: each number appears exactly once per column
3.   Row constraints: each number appears exactly once per row
4.   3x3 boxes constraints: each number appears exactly once per 3x3 block

## Part 2 (Coding)

Encode the above constraints and all the other ones as a CSP using z3py and solve the Sudoku instance given in the figure below. **Use only boolean variables** and do not use the **Distinct** function.

* Provide the code to solve that solves the given problem instance using z3py and only boolean variables. The code should output a reasonable visualization of the solution, for example printed in text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/sudoku1.jpg" width="400"/>



In [ ]:
# given problem:
puzzle = [
    [0,1,0, 4,0,2, 0,5,0],
    [5,0,0, 0,0,0, 0,0,6],
    [0,0,0, 3,0,1, 0,0,0],

    [7,0,5, 0,0,0, 4,0,8],
    [0,0,0, 0,0,0, 0,0,0],
    [2,0,8, 0,0,0, 5,0,9],

    [0,0,0, 9,0,6, 0,0,0],
    [6,0,0, 0,0,0, 0,0,2],
    [0,7,0, 1,0,3, 0,4,0],
]


In [ ]:
# z3py code to solve above Sudoku goes here. Do not use Int(*), only Bool(*)
from z3 import *

s = Solver()

# Constraints:  k -> number, i -> row, j -> column
X = [[[Bool(f"X_{k}_{i}_{j}") for j in range(9)] for i in range(9)] for k in range(9)]

for i in range(9):
    for j in range(9):
        num = puzzle[i][j]
        if num != 0:
            s.add(X[num - 1][i][j])

#print(s)

# 1. Each cell is either empty or has one value in [1..9] and the value appears only once:
# Check for only one number in each of the cells:
for i in range(9):
    for j in range(9):
        # At least one number in each cell:
        s.add(Or([X[k][i][j] for k in range(9)]))
        # At most one number in each ceel:
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                s.add(Not(And(X[k1][i][j], X[k2][i][j])))

# 2. Rows constraints: each number appear exactly once per row:
for k in range(9):
    for i in range(9):
        # At least once in the row:
        s.add(Or([X[k][i][j] for j in range(9)]))
        # At most once in the row:
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                s.add(Not(And(X[k][i][j1], X[k][i][j2])))

# 3. Column constraints: each number appears exactly once per column:
for k in range(9):
    for j in range(9):
        s.add(Or([X[k][i][j] for i in range(9)]))
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                s.add(Not(And(X[k][i1][j], X[k][i2][j])))

# 4. 3x3 boxes constraints: each number appears exactly once per 3x3 block:
for k in range(9):
    for box_i in range(3):
        for box_j in range(3):
            cells = []
            for di in range(3):
                for dj in range(3):
                    i = 3 * box_i + di
                    j = 3 * box_j + dj
                    cells.append(X[k][i][j])
            s.add(Or(cells))
            for idx1 in range(9):
                for idx2 in range(idx1 + 1, 9):
                    s.add(Not(And(cells[idx1], cells[idx2])))
#print(s)


In [ ]:
# Solution:
s.push()

if s.check() == sat:
    m = s.model()
    result = [[0 for _ in range(9)] for _ in range(9)]
    for i in range(9):
        for j in range(9):
            for k in range(9):
                if m.evaluate(X[k][i][j]):
                    result[i][j] = k + 1
    print("Solution:")
    for row in result:
        print(" ".join(str(num) for num in row))
else:
    print("No solution")
s.pop()

Solution:
3 1 9 4 6 2 8 5 7
5 2 4 7 9 8 1 3 6
8 6 7 3 5 1 9 2 4
7 3 5 2 1 9 4 6 8
1 9 6 8 4 5 2 7 3
2 4 8 6 3 7 5 1 9
4 5 3 9 2 6 7 8 1
6 8 1 5 7 4 3 9 2
9 7 2 1 8 3 6 4 5


### Uniqueness
Is your solution unique? Prove it with a Z3py solver or provide a second solution. **(5 points)**

In [ ]:
# To check if the solution we found is unique, we will add it as a constraint and try to finb another solution:

different_solution = []

for row in range(9):
    for col in range(9):
        number = result[row][col]
        # no the same assignment for the same cell:
        different_solution.append(Not(X[number - 1][row][col]))

s.push() #save the solution

s.add(Or(different_solution)) # at least one should be different

if s.check() == sat:
    print(" not unique:")
    m2 = s.model()
    for row in range(9):
        line = ""
        for col in range(9):
            for num in range(9):
                if m2.evaluate(X[num][row][col]):
                    line += str(num + 1) + " "
                    break
        print(line.strip())
else:
    print("the solution is unique (didn't find another one)")

# recover the previous constraints:
s.pop()


the solution is unique (didn't find another one)


### More Uniqueness
If you delete the 1 in the top left box of the Sudoku problem above, how many unique solutions are there? Hint: should be fewer than 5000. **(5 points)**

In [ ]:
puzzle_2 = [
    [0,0,0, 4,0,2, 0,5,0],
    [5,0,0, 0,0,0, 0,0,6],
    [0,0,0, 3,0,1, 0,0,0],

    [7,0,5, 0,0,0, 4,0,8],
    [0,0,0, 0,0,0, 0,0,0],
    [2,0,8, 0,0,0, 5,0,9],

    [0,0,0, 9,0,6, 0,0,0],
    [6,0,0, 0,0,0, 0,0,2],
    [0,7,0, 1,0,3, 0,4,0],
]

s_2 = Solver()

# Constraints:  k -> number, i -> row, j -> column
X = [[[Bool(f"X_{k}_{i}_{j}") for j in range(9)] for i in range(9)] for k in range(9)]

for i in range(9):
    for j in range(9):
        num = puzzle_2[i][j]
        if num != 0:
            s_2.add(X[num - 1][i][j])

# 1. Each cell is either empty or has one value in [1..9] and the value appears only once:
# Check for only one number in each of the cells:
for i in range(9):
    for j in range(9):
        # At least one number in each cell:
        s_2.add(Or([X[k][i][j] for k in range(9)]))
        # At most one number in each ceel:
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                s_2.add(Not(And(X[k1][i][j], X[k2][i][j])))

# 2. Rows constraints: each number appear exactly once per row:
for k in range(9):
    for i in range(9):
        # At least once in the row:
        s_2.add(Or([X[k][i][j] for j in range(9)]))
        # At most once in the row:
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                s_2.add(Not(And(X[k][i][j1], X[k][i][j2])))

# 3. Column constraints: each number appears exactly once per column:
for k in range(9):
    for j in range(9):
        s_2.add(Or([X[k][i][j] for i in range(9)]))
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                s_2.add(Not(And(X[k][i1][j], X[k][i2][j])))

# 4. 3x3 boxes constraints: each number appears exactly once per 3x3 block:
for k in range(9):
    for box_i in range(3):
        for box_j in range(3):
            cells = []
            for di in range(3):
                for dj in range(3):
                    i = 3 * box_i + di
                    j = 3 * box_j + dj
                    cells.append(X[k][i][j])
            s_2.add(Or(cells))
            for idx1 in range(9):
                for idx2 in range(idx1 + 1, 9):
                    s_2.add(Not(And(cells[idx1], cells[idx2])))


count = 0
max = 5000

# keep checking for unique solutions:
while s_2.check() == sat and count < max:
    model = s_2.model()

    # current solution:
    current_grid = [[0 for _ in range(9)] for _ in range(9)]
    for row in range(9):
        for col in range(9):
            for num in range(9):
                if is_true(model.evaluate(X[num][row][col])):
                    current_grid[row][col] = num + 1

    # block this solution so that it is not added again:
    different_cells = []
    for row in range(9):
        for col in range(9):
            num = current_grid[row][col]
            different_cells.append(Not(X[num - 1][row][col]))

    s_2.add(Or(different_cells))
    count += 1

print("Unique solutions:", count)


Unique solutions: 200


# Miracle Sudoku

Using z3py and only boolean variables, solve the Miracle Sudoku ([spoiler](https://www.youtube.com/watch?v=yKf9aUIxdb4)).
In this puzzle normal Sudoku rules apply in addition to the following rules:
* Any two cells separated by a knight's move from chess (moving forward two boxes and over one, in any direction) cannot contain the same digit (see image below).
* Any two cells separated by a king's move from chess (one box in any direction including diagonals) cannot contain the same digit.
* Any two orthogonally adjacent (up, down, left, or right) cells cannot contain consecutive digits.

### Knight's move constraint image
<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/knights.jpg" width="400"/>

The knight's move constraint would mean the yellow boxes cannot contain a 1.


## Miracle Sudoku Starting Information
Provide the code to solve the puzzle given below. The code should output a reasonable visualization of the solution, for example using text. **(10 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/miracle.jpg" width="400"/>

In [ ]:
# Solution code for Miracle Sudoku using z3py
puzzle_3 = [
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],

    [0,0,0, 0,0,0, 0,0,0],
    [0,0,1, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 2,0,0],

    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
]


s_3 = Solver()

# Constraints:  k -> number, i -> row, j -> column
X = [[[Bool(f"X_{k}_{i}_{j}") for j in range(9)] for i in range(9)] for k in range(9)]

for i in range(9):
    for j in range(9):
        num = puzzle_3[i][j]
        if num != 0:
            s_3.add(X[num - 1][i][j])

#print(s)

# 1. Each cell is either empty or has one value in [1..9] and the value appears only once:
# Check for only one number in each of the cells:
for i in range(9):
    for j in range(9):
        # At least one number in each cell:
        s_3.add(Or([X[k][i][j] for k in range(9)]))
        # At most one number in each ceel:
        for k1 in range(9):
            for k2 in range(k1 + 1, 9):
                s_3.add(Not(And(X[k1][i][j], X[k2][i][j])))

# 2. Rows constraints: each number appear exactly once per row:
for k in range(9):
    for i in range(9):
        # At least once in the row:
        s_3.add(Or([X[k][i][j] for j in range(9)]))
        # At most once in the row:
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                s_3.add(Not(And(X[k][i][j1], X[k][i][j2])))

# 3. Column constraints: each number appears exactly once per column:
for k in range(9):
    for j in range(9):
        s_3.add(Or([X[k][i][j] for i in range(9)]))
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                s_3.add(Not(And(X[k][i1][j], X[k][i2][j])))

# 4. 3x3 boxes constraints: each number appears exactly once per 3x3 block:
for k in range(9):
    for box_i in range(3):
        for box_j in range(3):
            cells = []
            for di in range(3):
                for dj in range(3):
                    i = 3 * box_i + di
                    j = 3 * box_j + dj
                    cells.append(X[k][i][j])
            s_3.add(Or(cells))
            for idx1 in range(9):
                for idx2 in range(idx1 + 1, 9):
                    s_3.add(Not(And(cells[idx1], cells[idx2])))

# New constraints for the s_3 solver:

# Knight's move constraints:
knight_moves = [(-2, -1),(-1, -2),(-2, 1),(-1, 2),(1, -2),(2, -1),(1, 2),(2, 1)]

for r in range(9):
    for c in range(9):
        for dr, dc in knight_moves:
            r2, c2 = r + dr, c + dc
            if 0 <= r2 < 9 and 0 <= c2 < 9:
                for d in range(9):
                    s_3.add(Or(Not(X[d][r][c]), Not(X[d][r2][c2])))

# Kings moves constraints:
king_moves = [(-1, -1),(-1, 0),(-1, 1),(0, -1),(0, 1),(1, -1),(1, 0),(1, 1)]

for r in range(9):
    for c in range(9):
        for dr, dc in king_moves:
            r2, c2 = r + dr, c + dc
            if 0 <= r2 < 9 and 0 <= c2 < 9:
                for d in range(9):
                    s_3.add(Or(Not(X[d][r][c]), Not(X[d][r2][c2])))

# Ortogonal constraints:
for r in range(9):
    for c in range(9):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            r2, c2 = r + dr, c + dc
            if 0 <= r2 < 9 and 0 <= c2 < 9:
                for d in range(9):
                    if d > 0:
                        s_3.add(Or(Not(X[d][r][c]), Not(X[d - 1][r2][c2])))
                    if d < 8:
                        s_3.add(Or(Not(X[d][r][c]), Not(X[d + 1][r2][c2])))

# solve:
if s_3.check() == sat:
    model = s_3.model()
    solution = [[0 for _ in range(9)] for _ in range(9)]
    for r in range(9):
        for c in range(9):
            for d in range(9):
                if is_true(model.evaluate(X[d][r][c])):
                    solution[r][c] = d + 1
    # print out the solution:
    for row in solution:
        print(' '.join(str(num) for num in row))
else:
    print("No solution")

4 8 3 7 2 6 1 5 9
7 2 6 1 5 9 4 8 3
1 5 9 4 8 3 7 2 6
8 3 7 2 6 1 5 9 4
2 6 1 5 9 4 8 3 7
5 9 4 8 3 7 2 6 1
3 7 2 6 1 5 9 4 8
6 1 5 9 4 8 3 7 2
9 4 8 3 7 2 6 1 5


---
# Number Challenge in Z3py

Write Z3py code to solve the Number Challenge puzzle shown below and print the solution found. You can use the `Int` type to create integer variables in z3py. **(20 points)**

<img src="https://raw.githubusercontent.com/stanleybak/CS7320-AI/master/Knowledge-based/chal.jpg" width="300"/>

In [ ]:
from z3 import *

# cells to fill:
positions = [
           (0,1), (0,2),
    (1,0), (1,1), (1,2), (1,3),
           (2,1), (2,2)
]

# Define the integer variables for each of the positions:
cells = {(r, c): Int(f"cell_{r}_{c}") for (r, c) in positions}

s_4 = Solver()

# Each variable is in [1..8]:
for var in cells.values():
    s_4.add(And(var >= 1, var <= 8))

# All values are unique:
cell_vars = list(cells.values())
for i in range(len(cell_vars)):
    for j in range(i + 1, len(cell_vars)):
        s_4.add(cell_vars[i] != cell_vars[j])

# Consecutive numbers cant be adjacent (sides and diagonals):
adjacent_offsets = [(-1,-1), (-1,0), (-1,1),
                    ( 0,-1),         ( 0,1),
                    ( 1,-1), ( 1,0), ( 1,1)]

for (r1, c1) in positions:
    for dr, dc in adjacent_offsets:
        r2, c2 = r1 + dr, c1 + dc
        if (r2, c2) in positions:
            # avoid consequtive numbers:
            s_4.add(Abs(cells[(r1,c1)] - cells[(r2,c2)]) != 1)

# solve:
if s_4.check() == sat:
    m = s_4.model()
    grid = [["." for _ in range(4)] for _ in range(3)]
    for (r, c) in positions:
        grid[r][c] = str(m[cells[(r,c)]].as_long())
    # print:
    for row in grid:
        print(" ".join(row))
else:
    print("No solution")


. 3 5 .
7 1 8 2
. 4 6 .





---


# Propositional Logic

Next, we will use truth tables and z3py to prove logical entailment.

## Truth Tables

Using markdown tables and bold to show relevant rows, use the truth table method to show logical truths.


### Example

Here's an example that shows $p, p \rightarrow q \models q$.  

| p | q | p -> q |
|-------|-------|---------------------|
| **T**     | **T**     | **T**                   |
| T     | F     | F                   |
| F     | T     | T                   |
| F     | F     | T                   |

Since $q$ is true whenever $p$ and $p \rightarrow q$ are true, the expression is true.

### Questions

Prove that the following semantic entailment expressions are true by using the truth table approach.  Bold the appropriate rows **(7 point)**

 * $(p\rightarrow q)\rightarrow r, s \rightarrow \neg p, t, \neg s, t \rightarrow q \models r$
 * $\emptyset \models q \rightarrow (p \rightarrow (p \rightarrow (q \rightarrow p)))$

| p | q | r | s | t | p-> q | (p -> q) -> r | ¬p | s-> ¬p | ¬s | t -> q | r |               |
|---|---|---|---|---|------|-------------|----|--------|-----|------|---|-----------------|
| T | T | T | T | T | T | T | F | F | F | T | T |               |
| T | T | T | T | F | T | T | F | F | F | T | T |               |
| **T** | **T** | **T** | **F** | **T** | **T** | **T** | **F** | **T** | **T** | **T** | **T** | <<<< this row |
| T | T | T | F | F | T | T | F | T | T | T | T |               |
| T | T | F | T | T | T | F | F | F | F | T | F |               |
| T | T | F | T | F | T | F | F | F | F | T | F |               |
| T | T | F | F | T | T | F | F | T | T | T | F |               |
| T | T | F | F | F | T | F | F | T | T | T | F |               |
| T | F | T | T | T | F | T | F | F | F | F | T |               |
| T | F | T | T | F | F | T | F | F | F | T | T |               |
| T | F | T | F | T | F | T | F | T | T | F | T |               |
| T | F | T | F | F | F | T | F | T | T | T | T |               |
| T | F | F | T | T | F | T | F | F | F | F | F |               |
| T | F | F | T | F | F | T | F | F | F | T | F |               |
| T | F | F | F | T | F | T | F | T | T | F | F |               |
| T | F | F | F | F | F | T | F | T | T | T | F |               |
| F | T | T | T | T | T | T | T | T | F | T | T |               |
| F | T | T | T | F | T | T | T | T | F | T | T |               |
| **F** | **T** | **T** | **F** | **T** | **T** | **T** | **T** | **T** | **T** | **T** | **T** | <<<< this row |
| F | T | T | F | F | T | T | T | T | T | T | T |               |
| F | T | F | T | T | T | F | T | T | F | T | F |               |
| F | T | F | T | F | T | F | T | T | F | T | F |               |
| F | T | F | F | T | T | F | T | T | T | T | F |               |
| F | T | F | F | F | T | F | T | T | T | T | F |               |
| F | F | T | T | T | T | T | T | T | F | F | T |               |
| F | F | T | T | F | T | T | T | T | F | T | T |               |
| F | F | T | F | T | T | T | T | T | T | F | T |               |
| F | F | T | F | F | T | T | T | T | T | T | T |               |
| F | F | F | T | T | T | F | T | T | F | F | F |               |
| F | F | F | T | F | T | F | T | T | F | T | F |               |
| F | F | F | F | T | T | F | T | T | T | F | F |               |
| F | F | F | F | F | T | F | T | T | T | T | F |               |


| p | q | q → (p -> (p -> (q -> p))) |
|---|---|--------------------------|
| **T** | **T** | **T** |
| **T** | **F** | **T** |
| **F** | **T** | **T** |
| **F** | **F** | **T** |




---


 ## Logical Entailment using z3py

 Use Z3py to prove the above two entailments are true using satisfiability. See section 7.5 in the book for details. **(8 points)**

In [ ]:
from z3 import *
# 1)
# propositional variables:
p, q, r, s, t = Bools('p q r s t')

premises = [
    Implies(Implies(p, q), r),
    Implies(s, Not(p)),
    t,
    Not(s),
    Implies(t, q)
]

# negate the conlcusion r to check satisfiability:
formula = And(*premises, Not(r))

solver = Solver()
solver.add(formula)

if solver.check() == unsat:
    print("Entailment holds")
else:
    print("Entailment doesnt hold")


Entailment holds


In [ ]:
# 2)

# propositional variables:
p, q = Bools('p q')

# expression to test:
expr = Implies(q, Implies(p, Implies(p, Implies(q, p))))

solver = Solver()
solver.add(Not(expr))

if solver.check() == unsat:
    print("Entailment holds")
else:
    print("Entailment doesnt hold")


Entailment holds
